In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import cv2

from PIL import Image
from matplotlib import pyplot as plt
from dotenv import load_dotenv

load_dotenv()
sns.color_palette('colorblind')
# plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

try:
    pc_dpi = int(os.getenv('DPI'))
except TypeError:
    pc_dpi = 100
if pc_dpi is None:
    pc_dpi = 100


# Data adjustment and augmentation :

## 1 - Size adjust for model input
- Adjusting the image to kick away the scale : that ensures that the model will not be trained on the scale position/label/text whatever - We have room to spare since the images are all high res.
- The adjusted images will be reduced to a more manageable size to be fed to the model (400px*400px), square crop focused on center.

## 2 - Simple 'augmentation' :
- The initial image aspect ratio is 4:3 @ 4K. That means the center crop will leave details on the left and right side of the image out of the output.
- We can both get that information and augment the size of our dataset while staying meaningful : making a sample on left and right sides of the image makes 1 sample 3 output images.
- There doesn't seem to be a constant scale between the images so we might as well get zooms. We  will base our sampling on 400*400px (input size for model), cropping while making sure the average pixel value of the crop is somewhat similar to the initial image.

## 3 - Data augmentation :
- Will run standard data augmentation techniques to enlarge the dataset while keeping it meaningful and keeping overfitting in mind
- Since the images

In [2]:
test_image_path = "../imgs/MA184.jpg"

test_image = Image.open(fp=test_image_path)


In [ ]:
display(test_image)


In [ ]:
# crop percentage
crop_percentage = 0.075

# Adjust box dimensions
width, height = test_image.size
left = int(width * crop_percentage)
upper = int(height * crop_percentage)
right = int(width * (1 - crop_percentage))
lower = int(height * (1 - crop_percentage))

crop_box = (left, upper, right, lower)
cropped_image = test_image.crop(crop_box)

display(cropped_image)


<i>Bye scale!</i>

In [ ]:
display(cropped_image.getbbox())


In [ ]:
def augment(images: Image.Image, output_nbr: int = 0, sub_divide: bool = False, augment_subs: bool = False) -> list:
    ...